In [1]:
import numpy as np
import keras
import os
import pickle
import pandas as pd
from keras import callbacks
import cv2
from scipy import ndimage as ndi
import PIL
from PIL import Image
from keras.models import load_model
from sklearn.utils import shuffle
from os import listdir
from keras.models import Sequential,load_model
import tensorflow
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report
from tensorflow.keras import regularizers
from skimage.color import rgb2gray
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [2]:
directory_root = "../input/damage-finding-dataset/dataset2"
print(os.listdir("../input/damage-finding-dataset/dataset2"))
print(len(listdir(directory_root)))

['brain_normal', 'Damage dataset']
2


In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, tuple((64, 64)),PIL.Image.ANTIALIAS)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        print(f"Processing {plant_folder} ...")
      

        for image in plant_disease_folder_list:
            image_directory = f"{directory_root}/{plant_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(convert_image_to_array(image_directory))
                label_list.append(plant_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
Processing brain_normal ...
Processing Damage dataset ...
[INFO] Image loading completed


In [5]:
image_size = len(image_list)

In [6]:
img_data = pd.DataFrame({'label':label_list})
print(img_data.head())
print(len(img_data))

          label
0  brain_normal
1  brain_normal
2  brain_normal
3  brain_normal
4  brain_normal
4000


In [7]:
img_data["labels_integer"]=None
#index of new column
index_labels_integer = img_data.columns.get_loc("labels_integer")
#index of species column
index_species = img_data.columns.get_loc("label")
#to assign numeric labels starting with 0 for the first species
k = 0 
for i in range(len(img_data)):
    if i == 0:
        img_data.iloc[i, index_labels_integer] = k #here, k == 0
    if i > 0:
        if img_data.iloc[i-1, index_species] == img_data.iloc[i, index_species]:
            img_data.iloc[i, index_labels_integer] = k
        else:
            k += 1
            img_data.iloc[i, index_labels_integer] = k
img_data.head(20)

,label,labels_integer
0,brain_normal,0
1,brain_normal,0
2,brain_normal,0
3,brain_normal,0
4,brain_normal,0
5,brain_normal,0
6,brain_normal,0
7,brain_normal,0
8,brain_normal,0
9,brain_normal,0


In [13]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0
y =  img_data['labels_integer']
Y_one_hot = tensorflow.keras.utils.to_categorical(y, num_classes=2)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(np_image_list, Y_one_hot , test_size=0.2, random_state = 42) 

In [50]:
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3]) 
num_classes = 2

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(num_classes, activation='sigmoid'))



model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.001, decay=1e-7),loss='binary_crossentropy',
              metrics=['accuracy'])

In [51]:
best_model_file = "damage_detection.h5"
best_model = ModelCheckpoint(best_model_file, monitor='accuracy', verbose=1, save_best_only=True)

print('Training model...')
results = model.fit(x_train, y_train, epochs=100, batch_size=16, callbacks=[best_model])
print('Traing finished.')


Training model...
Epoch 1/100
200/200 [==============================] - 9s 43ms/step - loss: 0.6195 - accuracy: 0.6406

Epoch 00001: accuracy improved from -inf to 0.64062, saving model to damage_detection.h5
Epoch 2/100
200/200 [==============================] - 9s 43ms/step - loss: 0.5675 - accuracy: 0.6866

Epoch 00002: accuracy improved from 0.64062 to 0.68656, saving model to damage_detection.h5
Epoch 3/100
200/200 [==============================] - 9s 43ms/step - loss: 0.5490 - accuracy: 0.6944

Epoch 00003: accuracy improved from 0.68656 to 0.69437, saving model to damage_detection.h5
Epoch 4/100
200/200 [==============================] - 9s 46ms/step - loss: 0.5418 - accuracy: 0.6913

Epoch 00004: accuracy did not improve from 0.69437
Epoch 5/100
200/200 [==============================] - 9s 43ms/step - loss: 0.5171 - accuracy: 0.7269

Epoch 00005: accuracy improved from 0.69437 to 0.72688, saving model to damage_detection.h5
Epoch 6/100
200/200 [==============================

In [52]:
model2=load_model('./damage_detection.h5')   
scores=model2.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

25/25 [==============================] - 1s 23ms/step - loss: 0.7423 - accuracy: 0.8238
Test Accuracy: 82.37500190734863


In [ ]:
predict_x=model.predict(x_test) 
classes_x=np.argmax(predict_x,axis=1)

In [ ]:
print(len(classes_x))